In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 r

In [1]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 4.2 MB/s eta 0:00:00


In [5]:
from langchain_core.language_models.llms import LLM
from typing import List
from groq import Groq
import os 

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
groq_api_key = user_secrets.get_secret("GROQ_API_KEY")

os.environ["GROQ_API_KEY"] = groq_api_key



class GroqLLM(LLM):
    model: str = "meta-llama/llama-4-scout-17b-16e-instruct"
    temperature: float = 1.0
    max_tokens: int = 1024

    def _call(self, prompt: str, stop: List[str] = None) -> str:
        client = Groq()  # assumes your env has the API key
        completion = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            max_completion_tokens=self.max_tokens,
            top_p=1,
            stream=False,
            stop=stop,
        )
        return completion.choices[0].message.content

    @property
    def _llm_type(self) -> str:
        return "custom-groq-llm"


In [12]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType

# Your tool
def calculator(x):
    return str(eval(x))

tools = [
    Tool(name="Calculator", func=calculator, description="Useful for basic math.")
]

# Initialize your custom LLM
llm = GroqLLM()

# Use it inside LangChain agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run
query = input()
response = agent.run(query)
print(response)


 5464+446464




> Entering new AgentExecutor chain...
Thought: I need to add two large numbers together, which is a basic arithmetic operation. I can use the Calculator tool to perform this calculation.

Action: Calculator
Action Input: 5464 + 446464
Observation: 451928
Thought:Thought: I now know the final answer

Final Answer: 451928

> Finished chain.
451928
